In [1]:
from tvb_multiscale.tvb_nest.nest_models.builders.nest_factory import load_nest, compile_modules

In [2]:
# compile_modules("izhikevich_hamker")

In [3]:
nest = load_nest()
nest.Install("izhikevich_hamkermodule")

2020-11-20 15:16:17,246 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.nest_factory - Loading a NEST instance...
2020-11-20 15:16:17,246 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.nest_factory - Loading a NEST instance...
2020-11-20 15:16:17,249 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.nest_factory - NEST_INSTALL_DIR: /Users/andre/opt/anaconda3/envs/dbs-env
2020-11-20 15:16:17,249 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.nest_factory - NEST_INSTALL_DIR: /Users/andre/opt/anaconda3/envs/dbs-env
2020-11-20 15:16:17,251 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.nest_factory - NEST_DATA_DIR: /Users/andre/opt/anaconda3/envs/dbs-env/share/nest
2020-11-20 15:16:17,251 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.nest_factory - NEST_DATA_DIR: /Users/andre/opt/anaconda3/envs/dbs-env/share/nest
2020-11-20 15:16:17,254 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.nest_factory - NEST_DOC_DIR: /Users/andre/opt/anaconda3/envs/d

NESTErrors.DynamicModuleManagementError: ("DynamicModuleManagementError in Install: Module 'izhikevich_hamkermodule' could not be opened.\nThe dynamic loader returned the following error: 'file not found'.\n\nPlease check LD_LIBRARY_PATH (OSX: DYLD_LIBRARY_PATH)!", 'DynamicModuleManagementError', <SLILiteral: Install>, ": Module 'izhikevich_hamkermodule' could not be opened.\nThe dynamic loader returned the following error: 'file not found'.\n\nPlease check LD_LIBRARY_PATH (OSX: DYLD_LIBRARY_PATH)!")

In [ ]:
# nest.Models()

In [ ]:
from copy import deepcopy
params_common = {"E_rev_AMPA": 0.0, "E_rev_GABA_A": -90.0, "V_th": 30.0, "c": -65.0,
                 "C_m": 1.0, "I_e": 0.0,
                 "t_ref": 10.0, "tau_rise": 1.0, "tau_rise_AMPA": 10.0, "tau_rise_GABA_A": 10.0,
                 "n0": 140.0, "n1": 5.0, "n2": 0.04}
paramsI = deepcopy(params_common)
paramsI.update({"a": 0.005, "b": 0.585, "d": 4.0})
paramsE = deepcopy(params_common)
paramsStr = deepcopy(params_common)
paramsStr.update({"V_th": 40.0, "C_m": 50.0,
                  "n0": 61.65, "n1": 2.59, "n2": 0.02,
                  "a": 0.05, "b": -20.0, "c": -55.0, "d": 377.0})

In [ ]:
exc1 = nest.Create("izhikevich_hamker", 1, params=paramsE)

In [ ]:
stimE = nest.Create("poisson_generator", 1, params={"rate": 10.0, "origin": 0.0, "start": 0.1})
nest.Connect(stimE, exc1, syn_spec={'weight': -1.0, 'delay': 1.0, "receptor_type": 0})

In [ ]:
record_from = ["V_m", "U_m", "I_syn", "I_syn_ex", "I_syn_in", "g_L", "g_AMPA", "g_GABA_A"]
n_vars = len(record_from)
exc1_mltr = nest.Create("multimeter", 1, params={"record_from": record_from, "interval": 1.0})
nest.Connect(exc1_mltr, exc1)
exc1_spikes_rec = nest.Create("spike_recorder", 1)
nest.Connect(exc1, exc1_spikes_rec)

In [ ]:
simulation_time = 10000.0 # ms
nest.Prepare()
nest.Run(simulation_time)

In [ ]:
from matplotlib import pyplot
pyplot.close("all")
exc1_spikes = nest.GetStatus(exc1_spikes_rec)[0]["events"]
rate = len(exc1_spikes["times"]) / simulation_time * 1000
f1 = pyplot.figure(figsize=(10, 2.5))
pyplot.plot(exc1_spikes["times"], exc1_spikes["senders"], ".")
pyplot.xlabel("Time (ms)")
pyplot.title("%g (Hz)" % rate)
pyplot.show()
exc1_ts = nest.GetStatus(exc1_mltr)[0]["events"]
f2, axes = pyplot.subplots(n_vars, 1, sharex=True, figsize=(10, 10))
for i_var, var in enumerate(record_from):
    varts = exc1_ts[var]
    ts = exc1_ts["times"]
    axes[i_var].plot(ts, varts)
    axes[i_var].set_title(var)
axes[-1].set_xlabel("Time (ms)")
pyplot.show()

In [ ]:
exc1_ts["V_m"].max()

In [ ]:
nest.GetDefaults("izhikevich_hamker")